## MonReader

In this project, images containing document pages being flipped or being held straight (no flipping) are used to train a Support Vector Machine model which is then used to predict if pages are being flipped or not in test images

In [1]:
import os
import shutil #file management module
import cv2 #openCV module
import pywt #wavelet transform module
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
#the following function copied from StackOverflow converts an image to a B&W image using wavelet transform. 

def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)   
    imArray /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [2]:
#define paths which contain training and testing images
training_data_path = './images/training/'
testing_data_path = './images/testing/'


In [3]:
#create a list of directories that contain TRAINING images
training_img_dirs = []
for entry in os.scandir(training_data_path):
    if entry.is_dir: #if entry is a directory
        training_img_dirs.append(entry.path) #append the directory path to the list

In [4]:
training_img_dirs

['./images/training/flip', './images/training/notflip']

In [6]:
#create a list of directories that containg TESTING images
testing_img_dirs = []

for entry in os.scandir(testing_data_path):
    if entry.is_dir:
        testing_img_dirs.append(entry.path)

In [7]:
testing_img_dirs

['./images/testing/flip', './images/testing/notflip']

In [17]:
# for the training data, create a list of arrays representing the stacked arrays os resized original images and 
# resized wavelet transformed images along with a list of labels (flip or nonflip)

X = [] # independent variable list
y = [] # dependent varibale list

for img_dir in training_img_dirs:
    for file in os.scandir(img_dir):
        img = cv2.imread(file.path) #read the image file
        scaled_img = cv2.resize(img, (32,32)) #resize the image
        wave_trans_img = w2d(img, 'db1', 5) #wavelet transform the original image
        scaled_wave_trans_img = cv2.resize(wave_trans_img, (32,32)) #resize the wavelet transformed image
        # now vertically stack the resized original image and resized wavelet transformed image
        combined_image = np.vstack((scaled_img.reshape(32*32*3,1),scaled_wave_trans_img.reshape(32*32,1)))
        X.append(combined_image)#append to the X list
        #create y list (dependent varibale)
        if 'notflip' in img_dir:
            y.append(0)
        else:
            y.append(1)
        
        

In [25]:
#create deep copies of X and y
X_train = X[:]
y_train = y[:]

In [28]:
X_train[0]

array([[118],
       [138],
       [143],
       ...,
       [  0],
       [  0],
       [  0]], dtype=uint8)

In [32]:
len(X_train[0])

4096

In [33]:
test = X_train[0].flatten()
len(test)

4096

In [27]:
len(X_train[0])

4096

In [38]:
# flatten the arrays for each image for training the SVC model
X_train_flat = np.array(X_train).reshape(len(X_train),4096).astype(float)
X_train_flat

array([[118., 138., 143., ...,   0.,   0.,   0.],
       [118., 138., 143., ...,   0.,   0.,   0.],
       [114., 139., 141., ...,   0.,   0.,   0.],
       ...,
       [153., 169., 168., ...,   3., 253.,   0.],
       [153., 169., 168., ..., 238.,   1., 254.],
       [153., 168., 169., ..., 222.,   1.,  96.]])

In [26]:
#create similar arrays of stacked images as above but now for the testing images as well as labels

X_test = [] # independent variable list
y_test = [] # dependent varibale list

for img_dir in testing_img_dirs:
    for file in os.scandir(img_dir):
        print(file.path) #print the file path just to keep track of the progress
        img = cv2.imread(file.path) #read the image file
        scaled_img = cv2.resize(img, (32,32)) #resize the image
        wave_trans_img = w2d(img, 'db1', 5) #wavelet transform the original image
        scaled_wave_trans_img = cv2.resize(wave_trans_img, (32,32))
        # now vertically stack the resized original image and resized wavelet transformed image
        combined_image = np.vstack((scaled_img.reshape(32*32*3,1),scaled_wave_trans_img.reshape(32*32,1)))
        X_test.append(combined_image) #append the stacked images to the list
        #create the y list
        if 'notflip' in img_dir:
            y_test.append(0)
        else:
            y_test.append(1)

./images/testing/flip\0001_000000020.jpg
./images/testing/flip\0002_000000012.jpg
./images/testing/flip\0002_000000013.jpg
./images/testing/flip\0002_000000015.jpg
./images/testing/flip\0002_000000017.jpg
./images/testing/flip\0003_000000005.jpg
./images/testing/flip\0003_000000008.jpg
./images/testing/flip\0003_000000022.jpg
./images/testing/flip\0003_000000025.jpg
./images/testing/flip\0004_000000006.jpg
./images/testing/flip\0004_000000013.jpg
./images/testing/flip\0004_000000014.jpg
./images/testing/flip\0004_000000027.jpg
./images/testing/flip\0005_000000003.jpg
./images/testing/flip\0005_000000008.jpg
./images/testing/flip\0005_000000018.jpg
./images/testing/flip\0005_000000023.jpg
./images/testing/flip\0005_000000028.jpg
./images/testing/flip\0006_000000001.jpg
./images/testing/flip\0006_000000006.jpg
./images/testing/flip\0006_000000008.jpg
./images/testing/flip\0006_000000012.jpg
./images/testing/flip\0006_000000021.jpg
./images/testing/flip\0006_000000022.jpg
./images/testing

./images/testing/flip\0046_000000012.jpg
./images/testing/flip\0046_000000013.jpg
./images/testing/flip\0046_000000016.jpg
./images/testing/flip\0046_000000018.jpg
./images/testing/flip\0046_000000019.jpg
./images/testing/flip\0046_000000023.jpg
./images/testing/flip\0046_000000025.jpg
./images/testing/flip\0047_000000010.jpg
./images/testing/flip\0047_000000015.jpg
./images/testing/flip\0047_000000021.jpg
./images/testing/flip\0047_000000026.jpg
./images/testing/flip\0048_000000010.jpg
./images/testing/flip\0048_000000011.jpg
./images/testing/flip\0049_000000017.jpg
./images/testing/flip\0049_000000019.jpg
./images/testing/flip\0049_000000023.jpg
./images/testing/flip\0049_000000025.jpg
./images/testing/flip\0049_000000027.jpg
./images/testing/flip\0050_000000023.jpg
./images/testing/flip\0050_000000024.jpg
./images/testing/flip\0051_000000021.jpg
./images/testing/flip\0051_000000022.jpg
./images/testing/flip\0052_000000017.jpg
./images/testing/flip\0052_000000018.jpg
./images/testing

./images/testing/notflip\0025_000000013.jpg
./images/testing/notflip\0025_000000018.jpg
./images/testing/notflip\0025_000000021.jpg
./images/testing/notflip\0025_000000022.jpg
./images/testing/notflip\0025_000000025.jpg
./images/testing/notflip\0026_000000004.jpg
./images/testing/notflip\0026_000000005.jpg
./images/testing/notflip\0026_000000010.jpg
./images/testing/notflip\0026_000000014.jpg
./images/testing/notflip\0026_000000024.jpg
./images/testing/notflip\0026_000000029.jpg
./images/testing/notflip\0027_000000001.jpg
./images/testing/notflip\0027_000000004.jpg
./images/testing/notflip\0027_000000010.jpg
./images/testing/notflip\0027_000000013.jpg
./images/testing/notflip\0027_000000017.jpg
./images/testing/notflip\0027_000000022.jpg
./images/testing/notflip\0027_000000023.jpg
./images/testing/notflip\0028_000000004.jpg
./images/testing/notflip\0028_000000024.jpg
./images/testing/notflip\0029_000000001.jpg
./images/testing/notflip\0029_000000003.jpg
./images/testing/notflip\0029_00

./images/testing/notflip\0056_000000011.jpg
./images/testing/notflip\0056_000000013.jpg
./images/testing/notflip\0056_000000022.jpg
./images/testing/notflip\0056_000000024.jpg
./images/testing/notflip\0056_000000027.jpg
./images/testing/notflip\0057_000000003.jpg
./images/testing/notflip\0057_000000007.jpg
./images/testing/notflip\0057_000000012.jpg
./images/testing/notflip\0057_000000023.jpg
./images/testing/notflip\0057_000000027.jpg
./images/testing/notflip\0058_000000002.jpg
./images/testing/notflip\0058_000000004.jpg
./images/testing/notflip\0058_000000013.jpg
./images/testing/notflip\0058_000000018.jpg
./images/testing/notflip\0058_000000020.jpg
./images/testing/notflip\0058_000000021.jpg
./images/testing/notflip\0058_000000025.jpg


In [39]:
X_test_flat = np.array(X_test).reshape(len(X_test),4096).astype(float)
X_test_flat

array([[110., 133., 135., ...,   0.,   0.,   0.],
       [124., 144., 149., ...,   0.,   0.,   0.],
       [124., 144., 149., ...,   0.,   0.,   0.],
       ...,
       [155., 169., 168., ...,   6., 251., 255.],
       [155., 169., 168., ...,   6., 251., 255.],
       [155., 169., 168., ...,   4., 249.,   0.]])

In [42]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler #StandardScaler was used in the video. I will use MinMax to scale values between 0 and 1
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline #see below for Pipeline creation
from sklearn.metrics import classification_report

In [43]:
#create a sklearn pipeline 
pipe = Pipeline([('scaler', MinMaxScaler()), ('svc', SVC(kernel = 'rbf', C = 10))]) #create a simple pipeline of scaling the data
# and then applying the SVC model with the specified hyperparameters (these hyperparameters will be tuned later on)

In [44]:
pipe.fit(X_train_flat, y_train)
pipe.score(X_test_flat, y_test)

1.0

In [46]:
y_predicted = pipe.predict(X_test_flat)

In [47]:
print(classification_report(y_test, y_predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       307
           1       1.00      1.00      1.00       290

    accuracy                           1.00       597
   macro avg       1.00      1.00      1.00       597
weighted avg       1.00      1.00      1.00       597



In [ ]:
# I am somehow getting a perfect score for predicting the flip(1) and nonflip(0) images!! 
# I will have to go back over the code to see if there is any data leak